<a href="https://colab.research.google.com/github/priontu/Developing-a-Flower-Predictor-using-Keras-on-Tensorflow-and-ImageNet-models/blob/main/flower_predictor_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
# dir(keras.applications) #Checking which engines are available/possible

In [ ]:
base_model = keras.applications.xception.Xception(weights = 'imagenet', include_top = True)

91884032/91884032 [==============================] - 1s 0us/step


In [ ]:
# base_model.summary()

In [ ]:
trainSet, info = tfds.load(name='oxford_flowers102', split='train+validation', as_supervised=True, with_info=True)
validSet = tfds.load(name='oxford_flowers102', split='test[90%:]', as_supervised=True)
testSet = tfds.load(name='oxford_flowers102', split='test[:90%]', as_supervised=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/6149 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteW8W41I/oxford_flowers102-test.tfrecord*...: …

Generating validation examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteW8W41I/oxford_flowers102-validation.tfrecord…

Dataset oxford_flowers102 downloaded and prepared to ~/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [ ]:
# info.features['label'].names

In [ ]:
# info

In [ ]:
nToAugment = 4
def augmentImages(image, label):
  resized_image = tf.image.resize_with_pad(image, 299, 299)
  imageL = [resized_image]
  myGen=keras.preprocessing.image.ImageDataGenerator(rotation_range=40, width_shift_range=[-.2,.2], brightness_range=[.6,1.0], shear_range=0.0, channel_shift_range=0.0, fill_mode='constant', cval=0.0, horizontal_flip=True, vertical_flip=True)
  augmented_images=[next(myGen.flow(resized_image)) for _ in range(nToAugment)]
  labels= [label.numpy() for _ in range(nToAugment+1)]
  imageL.extend(augmented_images)
  return imageL, labels

def augmentImagesTF(image, labels):
  func = tf.py_function(augmentImages, [image, labels], [tf.float32, tf.int32])
  return func

def mySqueeze(x, y):
  return tf.squeeze(x), y


In [ ]:
trainPipeAug = trainSet.batch(1).prefetch(1).map(augmentImagesTF, num_parallel_calls=16).unbatch().map(mySqueeze, num_parallel_calls=16).shuffle(512)

In [ ]:
cnt = 1
plt.figure(figsize=(32, 16))
for inst in trainPipeAug.take(4*8):
  plt.subplot(4,8,cnt)
  plt.imshow(inst[0].numpy()/255)
  plt.axis('off')
  plt.title(info.features['label'].names[inst[1][0]])
  cnt = cnt + 1

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-0d56d8a8f5b3>", line 3, in <module>
    for inst in trainPipeAug.take(4*8):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 766, in __next__
    return self._next_internal()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 752, in _next_internal
    output_shapes=self._flat_output_shapes)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 3014, in iterator_get_next
    "output_shapes", output_shapes)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
  

KeyboardInterrupt: ignored

<Figure size 2304x1152 with 0 Axes>

In [ ]:
def preproc(image, label):
  inp=keras.applications.xception.preprocess_input(image)
  return inp, label

def preprocessWithAspectRatio(image, label):
  resized_image=tf.image.resize_with_pad(image, 299, 299)
  final_image = keras.applications.xception.preprocess_input(resized_image)
  return final_image, label

trainPipe=trainSet.map(preprocessWithAspectRatio, num_parallel_calls=8).batch(128).prefetch(1)
validPipe = validSet.map(preprocessWithAspectRatio, num_parallel_calls=8).batch(128).prefetch(1)
testPipe = testSet.map(preprocessWithAspectRatio, num_parallel_calls=8).batch(128).prefetch(1)


In [ ]:
# base_model.layers[:-1]

In [ ]:
base_model = keras.applications.xception.Xception(weights = 'imagenet', include_top = False)

In [ ]:
# base_model.summary()

In [ ]:
base_model.output

In [ ]:
base_model.input

In [ ]:
base_model.inputs

In [ ]:
avg=keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(info.features['label'].num_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.inputs, outputs=output) # base_model.inputs or input?
# model.summary()

In [ ]:
for layer in model.layers[:-2]:
  layer.trainable = False

# for layer in model.layers:
#   print(layer.trainable)

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('oxFLowersMod-topFit.h5', save_best_only=True)

earlyStop_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights = True)
ss=5e-1
# ss = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=5.0,
#     decay_steps=10,
#     decay_rate=0.9)
# ss = 5
optimizer = keras.optimizers.SGD(learning_rate=ss)

model.compile(loss="sparse_categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])
model.fit(trainPipe, validation_data= validPipe, epochs = 25, callbacks=[checkpoint_cb, earlyStop_cb])


In [ ]:
for layer in model.layers[:-2]:
  layer.trainable = True

In [ ]:
trainSet, info = tfds.load(name = 'oxford_flowers102', split = 'train + validation', as_supervised = True, with_info = True)
validSet = tfds.load(name='oxford_flowers102', split='test[90%:]', as_supervised=True)
testSet = tfds.load(name='oxford_flowers102', split = 'test[:90%]', as_supervised=True)

def preprocessWithAspectRatio(image, label):
  resized_image=tf.image.resize_with_pad(image, 299, 299)
  final_image = keras.applications.xception.preprocess_input(resized_image)
  return final_image, label

trainPipe=trainSet.map(preprocessWithAspectRatio, num_parallel_calls=32).batch(32).prefetch(1)
validPipe = validSet.map(preprocessWithAspectRatio, num_parallel_calls=32).batch(32).prefetch(1)
testPipe = testSet.map(preprocessWithAspectRatio, num_parallel_calls=32).batch(32).prefetch(1)


In [ ]:
ss = 3e-2
# ss = 30
# ss = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=3,
#     decay_steps=10,
#     decay_rate=0.9)
f_path = "/content/gdrive/My Drive/flowerModel.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(f_path, save_best_only=True)

earlyStop_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights = True)
# ss=5e-1
# ss = 50
# optimizer = keras.optimizers.SGD(learning_rate=ss)

model.compile(loss="sparse_categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])


In [ ]:
model.fit(trainPipe, validation_data= validPipe, epochs = 50, callbacks=[checkpoint_cb, earlyStop_cb])

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from tensorflow import keras
# from preprocessDefinition import preprocess

def preprocessWithAspectRatio(image, label):
  resized_image=tf.image.resize_with_pad(image, 299, 299)
  final_image = keras.applications.xception.preprocess_input(resized_image)
  return final_image, label

preprocess = preprocessWithAspectRatio
f_path = "/content/gdrive/My Drive/oxFLowersMod-botFit.h5"
model=tf.keras.models.load_model(f_path)
evalset,info = tfds.load(name='oxford_flowers102', split='test',as_supervised=True,with_info=True)
evalPipe=evalset.map(preprocess,num_parallel_calls=16).batch(128).prefetch(1)
for feats,lab in evalPipe.unbatch().batch(2048).take(1):
  probPreds=model.predict(feats)
top1err=tf.reduce_mean(keras.metrics.sparse_top_k_categorical_accuracy(lab,probPreds,k=1))
top5err=tf.reduce_mean(keras.metrics.sparse_top_k_categorical_accuracy(lab,probPreds,k=5))
top10err=tf.reduce_mean(keras.metrics.sparse_top_k_categorical_accuracy(lab,probPreds,k=10))

64/64 [==============================] - 666s 10s/step


In [ ]:
top1err

<tf.Tensor: shape=(), dtype=float32, numpy=0.8911133>

In [ ]:
top5err

<tf.Tensor: shape=(), dtype=float32, numpy=0.9794922>

In [ ]:
top10err

<tf.Tensor: shape=(), dtype=float32, numpy=0.99316406>

In [ ]:
import numpy as np

IMG_PATH = ''
snapdragon = keras.preprocessing.image.load_img(IMG_PATH)

tigerLily = keras.preprocessing.image.load_img(IMG_PATH)

stemlessGentian = keras.preprocessing.image.load_img(IMG_PATH)

mexicanAster = keras.preprocessing.image.load_img(IMG_PATH)

iml = [snapdragon, tigerLily, stemlessGentian, mexicanAster]

keras.preprocessing

In [ ]:
Y_proba = model.predict(inputs)
topk = 10

for myProbs in ylcs = np.flip(np.argsort(myProbs))
  for i in range(0, topk):
    print(info.features['label'])
  print()
